<a href="https://colab.research.google.com/github/LuisPinheiro2611/NDVI/blob/main/indices_vegetacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import geemap
import ee
import geopandas as gpd

Obtenção dos índices espectrais de uma área de pastagens, por meio de imagens de satélites do Sentinel 2A, utilizando a plataforma GEE

In [7]:
ee.Authenticate()
ee.Initialize(project='ndvi-457802')

# Definir Área de Estudo
aoi=geemap.gdf_to_ee(gpd.read_file('/content/drive/MyDrive/projeto_fazendasAPC/Larga_das_caichoeiras/Area_imovel_EPSG4326.shp'))

# Função: indices de cada imagem, recortar cada imagem pela área de estudo definida
def indices(image):
  image=image.clip(aoi)
  indices={'ndvi':['B8','B4'],'ndre':['B8','B5'],'ndwi':['B3','B8'],}
  lista=[image.normalizedDifference(indices[key]).rename(key) for key in indices.keys()]

  evi=image.expression('2.5 * (( N - R ) / (( N ) + ( C1 * R ) - ( C2 * B ) + L ))',\
  {'N':image.select('B8'),'R':image.select('B4'),'B':image.select('B2'),'C1':6,'C2':7.5,'L':1}).rename('evi')

  savi=savi = image.expression(' (( N - R ) / ( N + R + L )) * (1 + L )',{ \
    'R':image.select('B4'),'N':image.select('B8'),'L':0.5}).rename('savi')

  vari = image.expression('( G - R ) / ( G + R - B )',{'R':image.select('B4'), \
    'B':image.select('B2'),'G':image.select('B3')}).rename('vari')

  lista=[image.normalizedDifference(indices[key]).rename(key) for key in indices.keys()]+[savi,vari,evi]

  return image.addBands(lista)

s2=ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(aoi).filterDate('2024-10-01','2025-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)).map(indices)

# Exibir uma imagem do NDVI da área
Map=geemap.Map()
Map.centerObject(aoi)
Map.addLayer(aoi)
Map.addLayer(s2.select('ndvi'),{'palette':['red','yellow','green'],min:-1,max:1},'NDVI')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…